In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [11]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_zixun_huanqiu=pd.DataFrame(columns=columns)
channel_zixun_huanqiu

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [10]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [12]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [13]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[12]/div/a/span').click()  #选择频道

In [14]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[12]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'资讯'

In [15]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').text
sub_channel

'环球'

In [16]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [38]:
print(datetime.datetime.now())

for month in range(5,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_zixun_huanqiu.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now())  
    


2020-12-17 21:13:47.268382


<ipython-input-38-2260673e8e73>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-38-2260673e8e73>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-38-2260673e8e73>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Tk4y167DX视频属性的抓取
BV号为1Tk4y167DX的视频属性抓取结束


<ipython-input-38-2260673e8e73>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为16v411z7qh视频属性的抓取
BV号为16v411z7qh的视频属性抓取结束
开始BV号为1BK4y1t7vp视频属性的抓取
BV号为1BK4y1t7vp的视频属性抓取结束
开始BV号为1SV411C7ti视频属性的抓取
BV号为1SV411C7ti的视频属性抓取结束
开始BV号为1Ba4y1e7U5视频属性的抓取
BV号为1Ba4y1e7U5的视频属性抓取结束
开始BV号为1u5411x7SY视频属性的抓取
BV号为1u5411x7SY的视频属性抓取结束
开始BV号为1oK4y147GZ视频属性的抓取
BV号为1oK4y147GZ的视频属性抓取结束
开始BV号为1NA411q7dR视频属性的抓取
BV号为1NA411q7dR的视频属性抓取结束
开始BV号为17C4y1p7n7视频属性的抓取
BV号为17C4y1p7n7的视频属性抓取结束
开始BV号为1q54y1X7tc视频属性的抓取
BV号为1q54y1X7tc的视频属性抓取结束
开始BV号为18A411q7PP视频属性的抓取
BV号为18A411q7PP的视频属性抓取结束
开始BV号为1mT4y1372x视频属性的抓取
BV号为1mT4y1372x的视频属性抓取结束
开始BV号为1Va4y1v7SF视频属性的抓取
BV号为1Va4y1v7SF的视频属性抓取结束
开始BV号为1xt4y1C7rZ视频属性的抓取
BV号为1xt4y1C7rZ的视频属性抓取结束
开始BV号为1HK411W7ZY视频属性的抓取
BV号为1HK411W7ZY的视频属性抓取结束
开始BV号为1xQ4y1P7ba视频属性的抓取
BV号为1xQ4y1P7ba的视频属性抓取结束
开始BV号为1w5411x7AS视频属性的抓取
BV号为1w5411x7AS的视频属性抓取结束
开始BV号为1Di4y147iZ视频属性的抓取
BV号为1Di4y147iZ的视频属性抓取结束
开始BV号为1Jv411z7xn视频属性的抓取
BV号为1Jv411z7xn的视频属性抓取结束
开始BV号为1gT4y1g7oe视频属性的抓取
BV号为1gT4y1g7oe的视频属性抓取结束
开始BV号为1qK4y1t7KK视频属性的抓取
BV号为1qK4y1t7KK的视频属性抓取结束
开始BV号为1Si4y147xk视频属性的抓取
BV号为1Si4y147xk的视

...

开始BV号为1kK411W76o视频属性的抓取
BV号为1kK411W76o的视频属性抓取结束
开始BV号为1Sk4y16736视频属性的抓取
BV号为1Sk4y16736的视频属性抓取结束
开始BV号为16v411z7oV视频属性的抓取
BV号为16v411z7oV的视频属性抓取结束
开始BV号为19C4y1H7Da视频属性的抓取
BV号为19C4y1H7Da的视频属性抓取结束
开始BV号为1qi4y147xc视频属性的抓取
BV号为1qi4y147xc的视频属性抓取结束
开始BV号为1Vi4y1x7pY视频属性的抓取
BV号为1Vi4y1x7pY的视频属性抓取结束
开始BV号为1sk4y1r782视频属性的抓取
BV号为1sk4y1r782的视频属性抓取结束
开始BV号为1st4y1y72r视频属性的抓取
BV号为1st4y1y72r的视频属性抓取结束
开始BV号为1FZ4y1H7Fz视频属性的抓取
BV号为1FZ4y1H7Fz的视频属性抓取结束
开始BV号为1fA411q7XA视频属性的抓取
BV号为1fA411q7XA的视频属性抓取结束
开始BV号为1gf4y1m7PF视频属性的抓取
BV号为1gf4y1m7PF的视频属性抓取结束
开始BV号为1Pp4y1D7Nz视频属性的抓取
BV号为1Pp4y1D7Nz的视频属性抓取结束
开始BV号为1LK4y1s7o4视频属性的抓取
BV号为1LK4y1s7o4的视频属性抓取结束
开始BV号为1Rp4y1D7pF视频属性的抓取
BV号为1Rp4y1D7pF的视频属性抓取结束
开始BV号为1CD4y1D7P3视频属性的抓取
BV号为1CD4y1D7P3的视频属性抓取结束
开始BV号为16z411i7FT视频属性的抓取
BV号为16z411i7FT的视频属性抓取结束
开始BV号为1h54y1B7zw视频属性的抓取
BV号为1h54y1B7zw的视频属性抓取结束
开始BV号为1UK411p7vL视频属性的抓取
BV号为1UK411p7vL的视频属性抓取结束
开始BV号为1mp4y1U7ps视频属性的抓取
BV号为1mp4y1U7ps的视频属性抓取结束
开始BV号为1iV411r7HK视频属性的抓取
BV号为1iV411r7HK的视频属性抓取结束
开始BV号为1wg4y1q78h视频属性的抓取
BV号为1wg4y1q78h的视

<ipython-input-38-2260673e8e73>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1HV411r7yU视频属性的抓取
BV号为1HV411r7yU的视频属性抓取结束
开始BV号为1Up4y1U7LU视频属性的抓取
BV号为1Up4y1U7LU的视频属性抓取结束
开始BV号为1fK411H7oF视频属性的抓取
BV号为1fK411H7oF的视频属性抓取结束
开始BV号为1Si4y1s7iY视频属性的抓取
BV号为1Si4y1s7iY的视频属性抓取结束
开始BV号为1Z54y1B77B视频属性的抓取
BV号为1Z54y1B77B的视频属性抓取结束
开始BV号为1kT4y1J7PH视频属性的抓取
BV号为1kT4y1J7PH的视频属性抓取结束
开始BV号为1Up4y1S7aK视频属性的抓取
BV号为1Up4y1S7aK的视频属性抓取结束
开始BV号为1Et4y1y7HM视频属性的抓取
BV号为1Et4y1y7HM的视频属性抓取结束
开始BV号为1Yg4y1i7Zw视频属性的抓取
BV号为1Yg4y1i7Zw的视频属性抓取结束
开始BV号为1Qg4y1v7GX视频属性的抓取
BV号为1Qg4y1v7GX的视频属性抓取结束
开始BV号为1KK411p7Xz视频属性的抓取
BV号为1KK411p7Xz的视频属性抓取结束
开始BV号为1Ji4y1x74E视频属性的抓取
BV号为1Ji4y1x74E的视频属性抓取结束
开始BV号为1fK4y1x7ng视频属性的抓取
BV号为1fK4y1x7ng的视频属性抓取结束
开始BV号为1wp4y1D7oJ视频属性的抓取
BV号为1wp4y1D7oJ的视频属性抓取结束
开始BV号为1254y1B7br视频属性的抓取
BV号为1254y1B7br的视频属性抓取结束
开始BV号为1BD4y1D7Xi视频属性的抓取
BV号为1BD4y1D7Xi的视频属性抓取结束
开始BV号为1g54y1B7Vf视频属性的抓取
BV号为1g54y1B7Vf的视频属性抓取结束
开始BV号为1r54y1B7m1视频属性的抓取
BV号为1r54y1B7m1的视频属性抓取结束
开始BV号为1zK4y1x76x视频属性的抓取
BV号为1zK4y1x76x的视频属性抓取结束
开始BV号为1Jt4y1y79y视频属性的抓取
BV号为1Jt4y1y79y的视频属性抓取结束
开始BV号为1af4y1271H视频属性的抓取
BV号为1af4y1271H的视

BV号为1Ui4y1u7eb的视频属性抓取结束
开始BV号为18D4y127kn视频属性的抓取
BV号为18D4y127kn的视频属性抓取结束
开始BV号为1Rp4y1v7qd视频属性的抓取
BV号为1Rp4y1v7qd的视频属性抓取结束
开始BV号为1CZ4y1M7hb视频属性的抓取
BV号为1CZ4y1M7hb的视频属性抓取结束
开始BV号为1wC4y1b78F视频属性的抓取
BV号为1wC4y1b78F的视频属性抓取结束
开始BV号为1CT4y1w7ha视频属性的抓取
BV号为1CT4y1w7ha的视频属性抓取结束
开始BV号为1wZ4y1M7v6视频属性的抓取
BV号为1wZ4y1M7v6的视频属性抓取结束
开始BV号为18i4y1M7AP视频属性的抓取
BV号为18i4y1M7AP的视频属性抓取结束
开始BV号为1bZ4y1M71D视频属性的抓取
BV号为1bZ4y1M71D的视频属性抓取结束
开始BV号为1Rt4y1U7Uw视频属性的抓取
BV号为1Rt4y1U7Uw的视频属性抓取结束
开始BV号为1264y1c73M视频属性的抓取
BV号为1264y1c73M的视频属性抓取结束
开始BV号为1dy4y1C7nT视频属性的抓取
BV号为1dy4y1C7nT的视频属性抓取结束
开始BV号为1w5411Y7Xh视频属性的抓取
BV号为1w5411Y7Xh的视频属性抓取结束
开始BV号为1ea4y1J7wM视频属性的抓取
BV号为1ea4y1J7wM的视频属性抓取结束
开始BV号为1e64y1c7h1视频属性的抓取
BV号为1e64y1c7h1的视频属性抓取结束
开始BV号为1q54y1e7c7视频属性的抓取
BV号为1q54y1e7c7的视频属性抓取结束
开始BV号为185411877B视频属性的抓取
BV号为185411877B的视频属性抓取结束
开始BV号为1mK4y1v7Lj视频属性的抓取
BV号为1mK4y1v7Lj的视频属性抓取结束
开始BV号为1Yk4y117SV视频属性的抓取
BV号为1Yk4y117SV的视频属性抓取结束
开始BV号为1wA411Y75a视频属性的抓取
BV号为1wA411Y75a的视频属性抓取结束
开始BV号为1DV411z7u8视频属性的抓取
BV号为1DV411z7u8的视频属性抓取结束
开始BV号为1xa4y1J79s

开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
开始BV号为1of4y1S7Gt视频属性的抓取
BV号为1of4y1S7Gt的视频属性抓取结束
开始BV号为1Q5411b7YA视频属性的抓取
BV号为1Q5411b7YA的视频属性抓取结束
开始BV号为1H54y1y7qT视频属性的抓取
BV号为1H54y1y7qT的视频属性抓取结束
开始BV号为1hK4y1e7Jk视频属性的抓取
BV号为1hK4y1e7Jk的视频属性抓取结束
开始BV号为1Hi4y1u7B6视频属性的抓取
BV号为1Hi4y1u7B6的视频属性抓取结束
开始BV号为1Jp4y1a7oL视频属性的抓取
BV号为1Jp4y1a7oL的视频属性抓取结束
开始BV号为13T4y1c7ag视频属性的抓取
BV号为13T4y1c7ag的视频属性抓取结束
开始BV号为16t4y1i721视频属性的抓取
BV号为16t4y1i721的视频属性抓取结束
开始BV号为18k4y1275r视频属性的抓取
BV号为18k4y1275r的视频属性抓取结束
开始BV号为1Dt4y1S74R视频属性的抓取
BV号为1Dt4y1S74R的视频属性抓取结束
开始BV号为1Rp4y1e7X9视频属性的抓取
BV号为1Rp4y1e7X9的视频属性抓取结束
开始BV号为1Ga4y1E72D视频属性的抓取
BV号为1Ga4y1E72D的视频属性抓取结束
开始BV号为1wy4y1y7wz视频属性的抓取
BV号为1wy4y1y7wz的视频属性抓取结束
开始BV号为18k4y1y7kY视频属性的抓取
BV号为18k4y1y7kY的视频属性抓取结束
开始BV号为1Sf4y1Q7EK视频属性的抓取
BV号为1Sf4y1Q7EK的视频属性抓取结束
开始BV号为1oz4y1o751视频属性的抓取
BV号为1oz4y1o751的视频属性抓取结束
开始BV号为1ty4y1y75a

In [37]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-37-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [39]:
channel_zixun_huanqiu.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
2月,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
3月,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56
4月,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96
5月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
6月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [40]:
channel_zixun_huanqiu.to_json('channel_zixun_huanqiu.json')

In [17]:
pd.set_option('display.max_rows', None)

In [27]:
channel_zixun_huanqiu[channel_zixun_huanqiu['时间']=='4月']

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1e54y1d7S1,4月,2020-04-07 11:35:14,央视频,机构认证,161.5万,1312,特朗普回应英国首相病情恶化：当被送到ICU 情况就非常严重了,资讯,环球,普通视频,...,"[资讯, 环球, 特朗普, 英国首相]",00:42,10.5万,6344,90,223,178,722,209,否
18t4y1U7Hy,4月,2020-04-09 12:24:03,环球网,机构认证,28万,2188,巨型集装箱船在韩国港口撞上码头，桥吊瞬间坍塌,资讯,环球,普通视频,...,"[韩国, 资讯, 环球, 环球网]",01:17,10.4万,4013,29,260,166,535,254,否
1LC4y1s7iT,4月,2020-04-08 16:52:35,环球网,机构认证,28万,2188,的确尴尬！美国市长刚呼吁市民待在家中，警察就在酒吧驱逐聚会时发现市长妻子,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 环球网]",00:34,10.3万,3458,28,182,40,315,103,否
1we41147S8,4月,2020-04-09 16:02:59,央视频,机构认证,161.5万,1312,【纽约医院ICU病房护士身穿垃圾袋】面对全副武装的CBS记者 她表示：我想要你的同款装备,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 纽约, 海外见闻, 新冠肺炎, 护士, 采访, 记者, 新型冠状病...",01:03,10.2万,6511,48,221,147,542,140,否
18p4y117if,4月,2020-04-02 10:32:35,央视频,机构认证,161.5万,1312,纽时记者：美国医疗人力不够？可以找中国啊,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 中国, 新冠肺炎, 纽约, 特朗普]",02:19,10.2万,7338,111,242,251,2106,3139,否
1Di4y1b7mQ,4月,2020-04-01 11:19:42,央视频,机构认证,161.5万,1312,医院外搭临时停尸房！央视记者探访疫情下纽约医院的“特殊行动”,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 医疗, 纽约, 海外见闻, 记者, 医院, 新型冠状病毒感染的肺炎]",03:23,10.1万,8050,178,346,238,661,723,否
1Ne411x7vH,4月,2020-04-03 14:34:40,环球网,机构认证,28万,2188,白宫：美国已进行130万次新冠病毒检测，只回收到一半检验结果,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 环球网]",00:27,10.1万,2253,21,157,265,313,35,否
1kz411b71k,4月,2020-04-10 21:06:18,环球网,机构认证,28万,2188,拜登支持团队发“特朗普vs福奇”视频怼特朗普：他的另类事实很危险！,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 拜登, 特朗普, 环球网]",01:50,7.1万,3268,86,386,174,335,92,否
19k4y197QD,4月,2020-04-05 12:26:11,央视频,机构认证,161.5万,1312,在纽约市接受新冠肺炎检测是啥体验？总台记者告诉你！,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 纽约, 海外见闻, 新冠肺炎, 医院, 新型冠状病毒感染的肺炎]",03:17,6.7万,6877,146,180,79,399,261,否
1Af4y1S747,4月,2020-04-16 21:24:13,央视频,机构认证,161.5万,1312,成为疫情最严重国家！ 白宫70天里都干了啥？,资讯,环球,普通视频,...,"[美国, 资讯, 环球, 白宫, 新冠肺炎, 特朗普]",03:07,6.5万,6441,494,424,271,371,202,否


In [30]:
channel_zixun_huanqiu=channel_zixun_huanqiu[:82]